### Imports

In [218]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors 
import matplotlib.cm
from numpy import linalg as LA
from ipywidgets import *
from manim import *

In [219]:
stop!

SyntaxError: invalid syntax (3319058519.py, line 1)

### Crossection color map animation


In [214]:
#%%manim -ql -v WARNING --disable_caching ColorMapCrossSection
#creates animation of crosssections fadind into each other

class ColorMapCrossSection(Scene):
    def construct(self):
        
        #data array goes in here shape= (x,y,z) where z is the time axis
        # Generate 3D array data with random integers between 1 and 5
        z_size = noframes
        ## mAnimation
        # Load the first image to display it
        #starting image
        run_time = 0.1 #speed of transitions
        #wait_time = 2 #time between transitions (not needed)
        image_mobject = ImageMobject("junk/slice_0.png").scale(3)  # Scale to desired size
        self.play(FadeIn(image_mobject),run_time=run_time)  # Show the first slice
        # self.wait(wait_time)

        # Transition through the z slices
        for z_slice in range(1, z_size):
            new_image_mobject = ImageMobject(f"junk/slice_{z_slice}.png").scale(3)
            self.play(Transform(image_mobject, new_image_mobject),run_time=run_time) 
            #self.wait(wait_time)  # Pause to observe each slice

        self.wait(2) #display end for a little lnger


In [215]:
data = np.load("data/figure11slices_full_r200_z100.npz")
#warning old data is rotated 90degrees

#creates the images (can be done outside of scene if correct naming is used)
# Function to generate a color map from 2D data and save it as an image
def create_colormap_image(data_slice, z_index):
    plt.imshow(data_slice, cmap='viridis', vmin=1, vmax=5,origin="lower",extent=paramspace.ravel())
    plt.axis('off')  # Remove axis
    # Save the plot as an image
    plt.savefig(f"junk/slice_{z_index}.png", bbox_inches='tight', pad_inches=0)
    plt.close()
paramspace = np.array([[-2,10],[-2,10]])

stack = np.stack([data[key] for key in data],axis=-1)
x_size, y_size, z_size = stack.shape
keys = data.keys()

# Create color map images for each slice along the z-axis
for z_slice in range(z_size):
    create_colormap_image(stack[:, :, z_slice], z_slice)
noframes = z_size
%manim -ql -v WARNING --disable_caching ColorMapCrossSection

Manim Community v0.18.1

### Orbit plot

#### Creates example orbit (old version)

In [217]:
#import needed: numpy, pyplot, plotLayout
dont run
def nd_orbit(fnct, x0, it):
    # calculates orbits of n dimensional map
    # Where:
    #- param fnct: Mapping function mapping x0 to x1 taking a single parameter
    #- param x0: Initial value
    #- param it: Number of iterations
    #- returns: The entire orbit, including the initial value
    #   Initialize
    orbit = np.zeros((len(x0),it))
    orbit[:,0]=x0
    #   Main loop
    for i in range(it-1):
        orbit[:,i+1] = fnct(orbit[:,i])
    return orbit

SyntaxError: invalid syntax (321627862.py, line 2)

In [9]:
def activation(x):
    #the actication  function f()
    #return (x+abs(x))/2 #ReLu
    return 1/(1+np.exp(-x)) #sigmoid 
    #return np.tanh(x) #tanh

In [10]:
#import needed: numpy, pyplot, plotLayout, nd_orbit
def neuron_2_discrete(x0,a,b,alpha ,Nit, ntrans):
    # calculates stable solution henon map
    # Where:
    #- param x0: Initial value
    #- param a: Parameters of Henon map, being [a,b] in the standard expression
    #- returns: An array containing the orbit of the Henon map without transients

    #   Initialize  
    linearmap = lambda xy: alpha * (a @ activation(xy) + b)
    n_iter = Nit
    # n_steps_until_stable = 100

    #   Calculate orbit
    orbit = nd_orbit(linearmap, x0, n_iter)
    orbit = orbit[:, ntrans+1:] #remove the transient (+1 to also remove initial condition)
    return orbit


In [11]:
def orbit_generate():
    x0 = -np.array([0.1,0.1])
    w11 = -10
    w12 = 5.9
    a = np.array([[w11,w12],
                [-6.6,0]])
    b = np.array([-3.8,3])
    alpha = 1
    orbit = neuron_2_discrete(x0,a,b,alpha,20,0)
    return orbit

#### Manimation

In [19]:
%%manim -ql -v WARNING --disable_caching plorbit

class plorbit(Scene):
    def calculate_axis_ranges(self, points):
        """
        Calculate the x and y axis ranges based on the min and max
        values of the orbit points.
        """
        # Extract x and y coordinates
        x_values = [h[0] for h in points]
        y_values = [h[1] for h in points]

        # Calculate min and max for x and y
        x_min, x_max = min(x_values), max(x_values)
        y_min, y_max = min(y_values), max(y_values)

        # Add padding to the axis ranges for better visualization
        padding_factor = 0.1  # 10% padding on each side
        x_padding = (x_max - x_min) * padding_factor
        y_padding = (y_max - y_min) * padding_factor

        # Return the padded ranges
        return x_min - x_padding, x_max + x_padding, y_min - y_padding, y_max + y_padding
    def animate(self,orbit):
        # Create axes for the orbit plot
        x_min, x_max, y_min, y_max = self.calculate_axis_ranges(orbit)

        # Create axes dynamically based on the calculated ranges
        axes = Axes(
            x_range=[x_min, x_max, (x_max - x_min) / 4],  # Divide into 4 intervals
            y_range=[y_min, y_max, (y_max - y_min) / 4],
            x_length=6,  # Length in units
            y_length=6,
            axis_config={"color": BLUE}
        )
        # axes = Axes(
        #     x_range=[-2, 2, 0.5], y_range=[-2, 2, 0.5],
        #     x_length=6, y_length=6,
        #     axis_config={"color": BLUE}
        # )
        labels = axes.get_axis_labels(x_label="h1", y_label="h2")
        numberplane = NumberPlane()
        self.add(axes, labels)
        #self.play(Create(axes), Write(labels))
        # Convert the simulated points to Manim points on the 2D plane
        orbit_points_manim = [
            axes.c2p(orbit[0,i],orbit[1,i]) for i in range(orbit.shape[1])
        ]
        # Create the orbit as a line with a moving dot
        orbit_path = VMobject(color=BLUE)
        orbit_path.set_points_as_corners(orbit_points_manim)
        moving_dot = Dot(orbit_points_manim[0], color=RED)

        # Animation: Draw the orbit path and move the dot along the path
        self.play(Create(orbit_path), FadeIn(moving_dot))
        self.play(MoveAlongPath(moving_dot, orbit_path), run_time=4, rate_func=linear)

        # Hold the final state for a moment
        self.wait(2)
    def construct(self):
        orbit_points = orbit_generate()
        self.animate(orbit_points)


Manim Community v0.18.1

### Better grid orbit evolution

#### Setup

In [172]:
#import needed: numpy, pyplot, plotLayout
def nd_orbit(fnct, x0, it):
    # calculates orbits of n dimensional map
    # Where:
    #- param fnct: Mapping function mapping x0 to x1 taking a single parameter
    #- param x0: Initial value
    #- param it: Number of iterations
    #- returns: The entire orbit, including the initial value
    #   Initialize
    orbit = np.zeros((it+1,len(x0)))
    orbit[0]=x0
    #   Main loop
    for i in range(1,it+1):
        orbit[i] = fnct(orbit[i-1])
    return orbit
def activation(x):
    #the actication  function f()
    #return (x+abs(x))/2 #ReLu
    #return 1/(1+np.exp(-x)) #sigmoid 
    return np.tanh(x) #tanh
active_tanh = lambda x: np.tanh(x)
active_sigma = lambda x:  1/(1+np.exp(-x))

#import needed: numpy, pyplot, plotLayout, nd_orbit
def neuron_orbiter(x0, weightM,bias ,Nit=100, ntrans=10, activation = activation):
    # calculates stable solution henon map
    # Where:
    #- param x0: Initial value
    #- param a: Parameters of Henon map, being [a,b] in the standard expression
    #- returns: An array containing the orbit of the Henon map without transients

    linearmap = lambda x: (weightM @ activation(x) + bias)

    #   Calculate orbit
    orbit = nd_orbit(linearmap, x0, Nit)
    orbit = orbit[ntrans:] #remove the transient (+1 to also remove initial condition)
    return orbit.T

In [211]:
#%%manim -ql -v WARNING --disable_caching GridOrbitAnimation

class GridOrbitAnimation(Scene):
    """Generates a grid of points coloured by location which then evolve discretly

    Args:
    !!need to be set outside of scene
        #axis params
        x_high = 1.5
        x_low= -x_high
        y_high = x_high
        y_low= -x_high
        xtick = int(np.round((x_high-x_low)/3,0))
        ytick = int(np.round((y_high-y_low)/3,0))
        
        steps = 80 #how many time steps to show
        display_axes  = False
        func = rotationM #/figure1chaos #the orbitfunction(x0)
        #animate it
        %manim -ql -v WARNING --disable_caching GridOrbitAnimation
    """
    def construct(self):
        viridis_cmap = matplotlib.colormaps['viridis'] #get color map
        def point_to_color(x, y):
            normed_value = norm(x+y) # Use both coordinates for color mapping
            color = viridis_cmap(normed_value)
            return list(color[:3]) #get rid of alpha channel
        #create axis even if not shown important for scaling
        x_range = y_range= [ x_low,x_high, xtick]
        y_range= [y_low, y_high, ytick]
        axis_length = 7
        axes = Axes(
            x_range=x_range,
            y_range=y_range,
            x_length=axis_length,
            y_length=axis_length,
            axis_config={"include_numbers": True}  # Display numbers on axes
        )
        if display_axes: self.play(Create(axes))

        #create grid of dots
        grid_size = 20  # number of points along each grid size
        x_coords =  y_coords = np.linspace(x_range[0], x_range[1], grid_size)
        y_coords = np.linspace(y_range[0], y_range[1], grid_size)
        dots_coord = [np.array([x,y]) for x in x_coords for y in x_coords]

        #create colors based on location
        normarray = x_coords+y_coords #for colors
        norm = matplotlib.colors.Normalize(vmin=normarray.min(), vmax=normarray.max())
        colors = [ManimColor.from_rgb(point_to_color(x, y)) for x in x_coords for y in x_coords]
        
        
        #calculate orbits for each dot
        orbits = []
        for dot_c in dots_coord:
            orbit = func(dot_c)
            #need to add 0 z coord for each coordinate
            orbits.append(np.concatenate([orbit, np.zeros((1,orbit.shape[1] ))]))
        
        manim_orbits = [axes.c2p(*(orbit)) for orbit in orbits] #scale onto manim axes
        #create actual dots at starting positions
        dots = VGroup(*[Dot(orbit[:,0],radius=0.05,color=colors[i]) for i ,orbit in enumerate(manim_orbits)])
        self.play(Create(dots))

        #move dots about to create animation
        num_steps = steps#30#orbits[0].shape[1]
        for t in range(num_steps):
            if t == 1: self.wait(0.5)
            for i, dot in enumerate(dots):
                #use c2p again to make sure correclty scaled
                dot.move_to(axes.c2p(*orbits[i][:,t]))  
            self.wait(0.2)  # pause to visualize each time step
    


#### Animate

In [200]:
def rotationM(x0):
    #a scaled rotation matrix (pretty)
    alpha = 1.3
    W = lambda theta: alpha * np.array([[np.cos(theta),np.sin(theta)],
                        [-np.sin(theta),np.cos(theta)]])
    b = np.array([0.,0.])
    orbit = neuron_orbiter(x0,W(np.pi/15),b,100,0,active_tanh)
    return orbit

def figure1chaos(x0):
    #chaotic behavoir from figure1 passman
    w11, w12 = -18,8
    w21 = -6
    W =  np.array([[w11,w12],
                    [w21,0]])
    b =  np.array([-3,4])

    orbit = neuron_orbiter(x0,W,b,100,0,active_sigma)
    return orbit

#to check params

# orbit = figure1chaos(np.array([0.1,0.1]))
# plt.scatter(orbit[0,:],orbit[1,:],c='k',s=1)
# plt.gca().set_aspect('equal')

In [210]:
#Generate Animation
#axis params
x_high = 1.5
x_low= -x_high
y_high = x_high
y_low= -x_high
xtick = int(np.round((x_high-x_low)/3,0))
ytick = int(np.round((y_high-y_low)/3,0))

steps = 80 #how many time steps to show
display_axes  = False
func = rotationM #/figure1chaos #the orbitfunction(x0)
#animate it
%manim -ql -v WARNING --disable_caching GridOrbitAnimation

Manim Community v0.18.1